# Zyfra Gold Mine

### The goal of our project is to model all the data collected through the stages of processing. Then, use the data to predict the amount of gold that should be recovered from gold ore. Finally, apply what we have learned from our model to optimize production and eliminate waste.

In [1]:
#!pip install scikit-learn
#!pip install joblib
#!pip install matplotlib
#!pip install seaborn

In [2]:
import pandas as pd 
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score, RandomizedSearchCV
from sklearn.metrics import accuracy_score, mean_squared_error, mean_absolute_error, make_scorer
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeRegressor
from sklearn.preprocessing import StandardScaler
#from sklearn.utils import shuffle
#from sklearn.preprocessing import OrdinalEncoder
import matplotlib.pyplot as plt
from scipy import stats as st
import seaborn as sns
import plotly_express as px

In [3]:
url1 = 'https://practicum-content.s3.us-west-1.amazonaws.com/datasets/gold_recovery_train.csv'
url2 = 'https://practicum-content.s3.us-west-1.amazonaws.com/datasets/gold_recovery_test.csv'
url3 = 'https://practicum-content.s3.us-west-1.amazonaws.com/datasets/gold_recovery_full.csv'
df = pd.read_csv(url1)
df1 = pd.read_csv(url2)
df2 = pd.read_csv(url3)

In [4]:
display(df.info())  
#display(df.describe()) 
display(df1.info())  
#display(df1.describe()) 
display(df2.info())  
#display(df2.describe()) 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16860 entries, 0 to 16859
Data columns (total 87 columns):
 #   Column                                              Non-Null Count  Dtype  
---  ------                                              --------------  -----  
 0   date                                                16860 non-null  object 
 1   final.output.concentrate_ag                         16788 non-null  float64
 2   final.output.concentrate_pb                         16788 non-null  float64
 3   final.output.concentrate_sol                        16490 non-null  float64
 4   final.output.concentrate_au                         16789 non-null  float64
 5   final.output.recovery                               15339 non-null  float64
 6   final.output.tail_ag                                16794 non-null  float64
 7   final.output.tail_pb                                16677 non-null  float64
 8   final.output.tail_sol                               16715 non-null  float64


None

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5856 entries, 0 to 5855
Data columns (total 53 columns):
 #   Column                                      Non-Null Count  Dtype  
---  ------                                      --------------  -----  
 0   date                                        5856 non-null   object 
 1   primary_cleaner.input.sulfate               5554 non-null   float64
 2   primary_cleaner.input.depressant            5572 non-null   float64
 3   primary_cleaner.input.feed_size             5856 non-null   float64
 4   primary_cleaner.input.xanthate              5690 non-null   float64
 5   primary_cleaner.state.floatbank8_a_air      5840 non-null   float64
 6   primary_cleaner.state.floatbank8_a_level    5840 non-null   float64
 7   primary_cleaner.state.floatbank8_b_air      5840 non-null   float64
 8   primary_cleaner.state.floatbank8_b_level    5840 non-null   float64
 9   primary_cleaner.state.floatbank8_c_air      5840 non-null   float64
 10  primary_clea

None

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22716 entries, 0 to 22715
Data columns (total 87 columns):
 #   Column                                              Non-Null Count  Dtype  
---  ------                                              --------------  -----  
 0   date                                                22716 non-null  object 
 1   final.output.concentrate_ag                         22627 non-null  float64
 2   final.output.concentrate_pb                         22629 non-null  float64
 3   final.output.concentrate_sol                        22331 non-null  float64
 4   final.output.concentrate_au                         22630 non-null  float64
 5   final.output.recovery                               20753 non-null  float64
 6   final.output.tail_ag                                22633 non-null  float64
 7   final.output.tail_pb                                22516 non-null  float64
 8   final.output.tail_sol                               22445 non-null  float64


None

In [5]:
# count for any duplicates
num_duplicates = df.duplicated().sum()
num_duplicates1 = df1.duplicated().sum()
num_duplicates2 = df2.duplicated().sum()
# print number of duplicate rows 
print(f"duplicate rows df: {num_duplicates}")
print(f"duplicate rows df1: {num_duplicates1}")
print(f"duplicate rows df2: {num_duplicates2}")

duplicate rows df: 0
duplicate rows df1: 0
duplicate rows df2: 0


### Our intial accesment there are a lot of missing bits of information.  There doesn't appaer to any one column missing more than ten percent, but most are missing something. 

In [6]:
# convert 'date' column to datetime
df['date'] = pd.to_datetime(df['date'])
df1['date'] = pd.to_datetime(df1['date'])
df2['date'] = pd.to_datetime(df2['date'])

In [7]:
df.describe()

,date,final.output.concentrate_ag,final.output.concentrate_pb,final.output.concentrate_sol,final.output.concentrate_au,final.output.recovery,final.output.tail_ag,final.output.tail_pb,final.output.tail_sol,final.output.tail_au,...,secondary_cleaner.state.floatbank4_a_air,secondary_cleaner.state.floatbank4_a_level,secondary_cleaner.state.floatbank4_b_air,secondary_cleaner.state.floatbank4_b_level,secondary_cleaner.state.floatbank5_a_air,secondary_cleaner.state.floatbank5_a_level,secondary_cleaner.state.floatbank5_b_air,secondary_cleaner.state.floatbank5_b_level,secondary_cleaner.state.floatbank6_a_air,secondary_cleaner.state.floatbank6_a_level
count,16860,16788.000000,16788.000000,16490.000000,16789.000000,15339.000000,16794.000000,16677.000000,16715.000000,16794.000000,...,16731.000000,16747.000000,16768.000000,16767.000000,16775.000000,16775.000000,16775.000000,16776.000000,16757.000000,16775.000000
mean,2017-05-02 03:04:06.401363968,4.716907,9.113559,8.301123,39.467217,67.213166,8.757048,2.360327,9.303932,2.687512,...,19.101874,-494.164481,14.778164,-476.600082,15.779488,-500.230146,12.377241,-498.956257,18.429208,-521.801826
min,2016-01-15 00:00:00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,-799.920713,0.000000,-800.021781,-0.423260,-799.741097,0.427084,-800.258209,0.024270,-810.473526
25%,2016-07-08 14:44:59,3.971262,8.825748,6.939185,42.055722,62.625685,7.610544,1.641604,7.870275,2.172953,...,14.508299,-500.837689,10.741388,-500.269182,10.977713,-500.530594,8.925586,-500.147603,13.977626,-501.080595
50%,2017-05-02 05:29:59,4.869346,10.065316,8.557228,44.498874,67.644601,9.220393,2.453690,10.021968,2.781132,...,19.986958,-499.778379,14.943933,-499.593286,15.998340,-499.784231,11.092839,-499.933330,18.034960,-500.109898
75%,2018-02-23 20:14:59,5.821176,11.054809,10.289741,45.976222,72.824595,10.971110,3.192404,11.648573,3.416936,...,24.983961,-494.648754,20.023751,-400.137948,20.000701,-496.531781,15.979467,-498.418000,24.984992,-499.565540
max,2018-08-18 10:59:59,16.001945,17.031899,18.124851,53.611374,100.000000,19.552149,6.086532,22.317730,9.789625,...,60.000000,-127.692333,28.003828,-71.472472,63.116298,-275.073125,39.846228,-120.190931,54.876806,-39.784927
std,NaN,2.096718,3.389495,3.825760,13.917227,11.960446,3.634103,1.215576,4.263208,1.272757,...,6.883163,84.803334,5.999149,89.381172,6.834703,76.983542,6.219989,82.146207,6.958294,77.170888


### Since there is some much information to contend with, we don't want to get lost in the weeds....or something about forest and trees.  We are less concerned with secondary data columns, our primary focus are the final outputs of heavy metals (Au, Ag, Pb).  Most of our column above the mean and median values are close, however the final ouput AU we see 39 for mean and 44 median, the results here are slightly skewed.  The best approach will then be to replace our missing values with the column's medium value.  

In [8]:
def replace_median(df):
    for column in df.columns:
        if df[column].isna().sum() > 0:  # Check missing values
            median_value = df[column].median()  # calculate median 
            df[column] = df[column].fillna(median_value)  # replace
    return df

# Apply the function
df = replace_median(df)
df1 = replace_median(df1)
df2 = replace_median(df2)

In [9]:
display(df.info())  
#display(df.describe()) 
display(df1.info())  
#display(df1.describe()) 
display(df2.info())  
#display(df2.describe()) 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16860 entries, 0 to 16859
Data columns (total 87 columns):
 #   Column                                              Non-Null Count  Dtype         
---  ------                                              --------------  -----         
 0   date                                                16860 non-null  datetime64[ns]
 1   final.output.concentrate_ag                         16860 non-null  float64       
 2   final.output.concentrate_pb                         16860 non-null  float64       
 3   final.output.concentrate_sol                        16860 non-null  float64       
 4   final.output.concentrate_au                         16860 non-null  float64       
 5   final.output.recovery                               16860 non-null  float64       
 6   final.output.tail_ag                                16860 non-null  float64       
 7   final.output.tail_pb                                16860 non-null  float64       
 8   final.

None

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5856 entries, 0 to 5855
Data columns (total 53 columns):
 #   Column                                      Non-Null Count  Dtype         
---  ------                                      --------------  -----         
 0   date                                        5856 non-null   datetime64[ns]
 1   primary_cleaner.input.sulfate               5856 non-null   float64       
 2   primary_cleaner.input.depressant            5856 non-null   float64       
 3   primary_cleaner.input.feed_size             5856 non-null   float64       
 4   primary_cleaner.input.xanthate              5856 non-null   float64       
 5   primary_cleaner.state.floatbank8_a_air      5856 non-null   float64       
 6   primary_cleaner.state.floatbank8_a_level    5856 non-null   float64       
 7   primary_cleaner.state.floatbank8_b_air      5856 non-null   float64       
 8   primary_cleaner.state.floatbank8_b_level    5856 non-null   float64       
 9   primary_

None

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22716 entries, 0 to 22715
Data columns (total 87 columns):
 #   Column                                              Non-Null Count  Dtype         
---  ------                                              --------------  -----         
 0   date                                                22716 non-null  datetime64[ns]
 1   final.output.concentrate_ag                         22716 non-null  float64       
 2   final.output.concentrate_pb                         22716 non-null  float64       
 3   final.output.concentrate_sol                        22716 non-null  float64       
 4   final.output.concentrate_au                         22716 non-null  float64       
 5   final.output.recovery                               22716 non-null  float64       
 6   final.output.tail_ag                                22716 non-null  float64       
 7   final.output.tail_pb                                22716 non-null  float64       
 8   final.

None

### Now, our date column is in the correct format.  We verified there are no duplicate rows.  Finally any missing data has been replaced with the median value. 

# Recovery Calculation

In [10]:
# calculate recovery
c = 'rougher.output.concentrate_au'
f = 'rougher.input.feed_au'
t = 'rougher.output.tail_au'
recovery_col = 'rougher.output.recovery'

new_df = df[[c, f, t]].copy()
# prevent dividing by zero
new_df = new_df[(new_df[c] != 0) & (new_df[f] != 0) & (new_df[t] != 0)]
# calculate recovery 
new_df['calculated recovery'] = (new_df[c] * (new_df[f] - new_df[t])) / (new_df[f] * (new_df[c] - new_df[t])) * 100
# eliminate outliers 
new_df = new_df[(new_df['calculated recovery'] >= 0) & (new_df['calculated recovery'] <= 100)]

# add the recovery column
new_df['recovery'] = df[recovery_col]

display(new_df.describe())

,rougher.output.concentrate_au,rougher.input.feed_au,rougher.output.tail_au,calculated recovery,recovery
count,15020.000000,15020.000000,15020.000000,15020.000000,15020.000000
mean,19.401468,7.706576,1.756404,84.689278,84.348832
std,3.946223,2.313123,0.702028,9.039300,8.300066
min,0.010000,0.010000,0.020676,0.045295,1.075750
25%,18.397162,6.506514,1.347830,80.506692,80.687905
50%,19.948804,7.597017,1.747907,85.757513,85.235997
75%,21.449102,9.107783,2.143336,90.779138,89.896002
max,28.522811,14.093363,9.688980,100.000000,100.000000


In [11]:
# plot recovery
fig = px.histogram(new_df, x='recovery', nbins=30, title='Distribution Recovery', color_discrete_sequence=['#00b3b3'])
fig.update_layout(
    xaxis_title='Recovery',
    yaxis_title='Frequency',
    showlegend=False,
    yaxis=dict(range=[0, 5000]))
fig.update_traces(marker=dict(line=dict(color='black', width=1.5)))

fig.show()
# plot Calculate Recovery
fig = px.histogram(new_df, x='calculated recovery', nbins=30, title='Distribution of Calculated Recovery', color_discrete_sequence=['#00b3b3'])
fig.update_layout(
    xaxis_title='Calculated Recovery',
    yaxis_title='Frequency',
    showlegend=False,
    yaxis=dict(range=[0, 5000]))
fig.update_traces(marker=dict(line=dict(color='black', width=1.5)))

fig.show()

### Our calculated recovery closely resembels the actual values.  The one difference is the actual values are largely centered around 85. Where our calculated, being multiplied and divided by each other, have averaged out over a wider area. 

In [12]:
# Calculate the MAE
mae = mean_absolute_error(new_df['recovery'], new_df['calculated recovery'])
print("MAE:", mae)

MAE: 0.678164121187856


### We had to do a lot to prepare the data for comparison, especially dealing with very small numbers that can easily throw off calculations when multiplying and dividing. The problem is further complicated because the result should be a portion of a whole number. By eliminating outlying values greater than 100 and less than zero, we were able to achieve a relatively good Mean Absolute Error of 0.68.

# Analyze the data

In [13]:
columns_of_interest = ['rougher.input.feed_pb','rougher.output.concentrate_pb','primary_cleaner.output.concentrate_pb','final.output.concentrate_pb',
'rougher.output.tail_pb','primary_cleaner.output.tail_pb',  'secondary_cleaner.output.tail_pb','final.output.tail_pb']

labels = ['rougher.input.feed', 'rougher.output.concentrate', 'primary_cleaner.output.concentrate', 'final.output.concentrate', 'rougher.output.tail', 'primary_cleaner.output.tail', 'secondary_cleaner.output.tail', 'final.output.tail']
# data for graph
means = {col: df[col].mean() for col in columns_of_interest}
plot_df = pd.DataFrame(list(means.items()), columns=['column', 'mean'])

# bar graph
fig = px.bar(plot_df, x='column', y='mean', color='mean', color_continuous_scale='greys', 
title='Average Lead Concentration at Different Stages')
fig.update_layout(yaxis_title='Concentration', xaxis_title='')
fig.update_traces(marker_line_color='black', marker_line_width=1.5)
fig.update_xaxes(tickvals=columns_of_interest, ticktext=labels)
fig.show()

### We see the concentration of lead incraese throughout each stage of processing.  This occuring probably has more to do due to it's similarities to gold (density), than an actual desire to process it. So the increase we see in the tailings probably is not a bad thing. 

In [14]:
columns_of_interest = ['rougher.input.feed_ag','rougher.output.concentrate_ag','primary_cleaner.output.concentrate_ag','final.output.concentrate_ag',
'rougher.output.tail_ag','primary_cleaner.output.tail_ag',  'secondary_cleaner.output.tail_ag','final.output.tail_ag']

labels = ['rougher.input.feed', 'rougher.output.concentrate', 'primary_cleaner.output.concentrate', 'final.output.concentrate', 'rougher.output.tail', 'primary_cleaner.output.tail', 'secondary_cleaner.output.tail', 'final.output.tail']
# data for graph
means = {col: df[col].mean() for col in columns_of_interest}
plot_df = pd.DataFrame(list(means.items()), columns=['column', 'mean'])


# bar graph
silver = ['#C0C0C0', '#DCDCDC', '#A9A9A9']
fig = px.bar(plot_df, x='column', y='mean', color='mean', color_continuous_scale=silver, 
title='Average Silver Concentration at Different Stages')
fig.update_layout(yaxis_title='Concentration', xaxis_title='')
fig.update_traces(marker_line_color='black', marker_line_width=1.5)
fig.update_xaxes(tickvals=columns_of_interest, ticktext=labels)
fig.show()

### The concentration of silver also declines throughout processing. While silver is a precious metal, it can be recovered from the tailing later in a secondary process if the concentrations are high enough to be economically feasible.

In [15]:
columns_of_interest = ['rougher.input.feed_au','rougher.output.concentrate_au','primary_cleaner.output.concentrate_au','final.output.concentrate_au',
'rougher.output.tail_au','primary_cleaner.output.tail_au',  'secondary_cleaner.output.tail_au','final.output.tail_au']

labels = ['rougher.input.feed', 'rougher.output.concentrate', 'primary_cleaner.output.concentrate', 'final.output.concentrate', 'rougher.output.tail', 'primary_cleaner.output.tail', 'secondary_cleaner.output.tail', 'final.output.tail']
# data for graph
means = {col: df[col].mean() for col in columns_of_interest}
plot_df = pd.DataFrame(list(means.items()), columns=['column', 'mean'])

# bar graph
gold = ['#D4AF37', '#FFD700', '#FFC200']
fig = px.bar(plot_df, x='column', y='mean', color='mean', color_continuous_scale= gold, 
title='Average Gold Concentration at Different Stages')
fig.update_layout(yaxis_title='Concentration', xaxis_title='')
fig.update_traces(marker_line_color='black', marker_line_width=1.5)
fig.update_xaxes(tickvals=columns_of_interest, ticktext=labels)
fig.show()

### Efficiently recovering the greatest percentage of gold is the main purpose of our processing. Looking at the chart, we are incrementally improving concentrations at each step while minimizing our losses in tailings.

# Preprocess Review of Data

In [16]:
# feed size training
df_feed_size = pd.DataFrame({'rougher.input.feed_size': df['rougher.input.feed_size'],
'primary_cleaner.input.feed_size': df['primary_cleaner.input.feed_size']})
# feed size test
df_feed_size1 = pd.DataFrame({'rougher.input.feed_size': df1['rougher.input.feed_size'],
'primary_cleaner.input.feed_size': df1['primary_cleaner.input.feed_size']})

print(df_feed_size.describe())
print(df_feed_size1.describe())

       rougher.input.feed_size  primary_cleaner.input.feed_size
count             16860.000000                     16860.000000
mean                 58.563359                         7.297285
std                  23.635551                         0.619166
min                   9.659576                         1.080000
25%                  47.597875                         6.940000
50%                  54.104257                         7.285000
75%                  64.587648                         7.700000
max                 484.967466                        10.470000
       rougher.input.feed_size  primary_cleaner.input.feed_size
count              5856.000000                      5856.000000
mean                 55.915236                         7.264651
std                  22.684428                         0.611526
min                   0.046369                         5.650000
25%                  43.898467                         6.885625
50%                  50.002004          

### When comparing feed size between our training and test datasets, we noticed some slight differences in mean and median values for rough input. Originally, we didn't see these differences as being a significant issue given that the standard deviation was still similar. However, as we went through the process of perparing our data for modeling there were too many challanges in utilizing the samples provided: zero values, negative values, conflicting sets of informations (columns). So we decided to ustilize the full data set to allow us to train the best model possible. 

In [17]:
# columns rough
columns = df2.iloc[:, 31:53]
display(columns.describe())

,rougher.calculation.floatbank10_sulfate_to_au_feed,rougher.calculation.floatbank11_sulfate_to_au_feed,rougher.calculation.au_pb_ratio,rougher.input.feed_ag,rougher.input.feed_pb,rougher.input.feed_rate,rougher.input.feed_size,rougher.input.feed_sol,rougher.input.feed_au,rougher.input.floatbank10_sulfate,...,rougher.input.floatbank11_xanthate,rougher.output.concentrate_ag,rougher.output.concentrate_pb,rougher.output.concentrate_sol,rougher.output.concentrate_au,rougher.output.recovery,rougher.output.tail_ag,rougher.output.tail_pb,rougher.output.tail_sol,rougher.output.tail_au
count,2.271600e+04,2.271600e+04,22716.000000,22716.000000,22716.000000,22716.000000,22716.000000,22716.000000,22716.000000,22716.000000,...,22716.000000,22716.000000,22716.000000,22716.000000,22716.000000,22716.000000,22716.000000,22716.000000,22716.000000,22716.000000
mean,3.558335e+03,2.295994e+03,2.379139,8.066737,3.307033,443.020133,57.871157,33.327915,7.567228,11.796550,...,5.969736,10.878412,6.903546,26.123002,17.888700,83.499096,5.608557,0.593608,18.045252,1.819673
std,6.110723e+04,4.173848e+04,0.809766,3.118539,1.439172,154.986489,23.423615,11.410296,3.020423,3.974492,...,1.215583,4.368878,2.801233,10.102059,6.776878,13.658939,1.046802,0.314614,3.254829,0.652436
min,-2.023459e+06,-4.014502e+06,-0.010548,0.000000,0.000000,0.000044,0.046369,0.000000,0.000000,0.000002,...,0.000010,0.000000,0.000000,0.000000,0.000000,0.000000,0.594562,0.000000,10.014648,0.020676
25%,2.506514e+03,2.501857e+03,2.038151,6.950419,2.669586,399.330406,46.697024,32.623484,6.492090,9.997432,...,5.490174,10.134209,6.381061,25.864578,17.940827,81.975763,5.052815,0.428550,16.133068,1.475072
50%,2.940838e+03,2.897014e+03,2.286493,8.302613,3.432054,488.968597,53.405122,36.559955,7.884832,11.998801,...,6.034551,11.785127,7.572855,29.382638,20.003202,86.144121,5.759630,0.590746,18.082595,1.808567
75%,3.611383e+03,3.528910e+03,2.670107,10.127245,4.193378,544.870965,63.807319,39.606272,9.659693,14.004933,...,6.700313,13.605763,8.472704,31.975308,21.555357,89.378541,6.306823,0.762431,19.781623,2.144160
max,6.211433e+06,1.537564e+06,39.379272,14.869652,7.142594,717.508837,484.967466,53.477742,14.093363,36.118275,...,18.709478,24.480271,18.394042,40.089993,28.824507,100.000000,12.719177,3.778064,66.117188,9.688980


### When visually inspected the stats of each column above you can see some of the issues in modeling.  Mostly, minimum values that are negative and occasionally maximum values that are several times over the 75 percentile (ex. rougher.calculation.au_pb_ratio).


# Preprocess


In [18]:
# remove outliers
def remove_outliers_iqr_all_columns(df):
    for column in df.columns:
        Q1 = df[column].quantile(0.05)
        Q3 = df[column].quantile(0.95)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        df = df[(df[column] >= lower_bound) & (df[column] <= upper_bound)]
    return df

# clean full data set
df2_cleaned = remove_outliers_iqr_all_columns(df2)

# review results
display(df2_cleaned.info())

<class 'pandas.core.frame.DataFrame'>
Index: 18309 entries, 0 to 22715
Data columns (total 87 columns):
 #   Column                                              Non-Null Count  Dtype         
---  ------                                              --------------  -----         
 0   date                                                18309 non-null  datetime64[ns]
 1   final.output.concentrate_ag                         18309 non-null  float64       
 2   final.output.concentrate_pb                         18309 non-null  float64       
 3   final.output.concentrate_sol                        18309 non-null  float64       
 4   final.output.concentrate_au                         18309 non-null  float64       
 5   final.output.recovery                               18309 non-null  float64       
 6   final.output.tail_ag                                18309 non-null  float64       
 7   final.output.tail_pb                                18309 non-null  float64       
 8   final.outpu

None

In [19]:
# drop the date column
df2_cleaned = df2_cleaned.drop(columns=['date'], errors='ignore')

# drop columns with any negative values
df2_cleaned = df2_cleaned.loc[:, (df2_cleaned >= 0).all()]
df2_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
Index: 18309 entries, 0 to 22715
Data columns (total 61 columns):
 #   Column                                    Non-Null Count  Dtype  
---  ------                                    --------------  -----  
 0   final.output.concentrate_ag               18309 non-null  float64
 1   final.output.concentrate_pb               18309 non-null  float64
 2   final.output.concentrate_sol              18309 non-null  float64
 3   final.output.concentrate_au               18309 non-null  float64
 4   final.output.recovery                     18309 non-null  float64
 5   final.output.tail_ag                      18309 non-null  float64
 6   final.output.tail_pb                      18309 non-null  float64
 7   final.output.tail_sol                     18309 non-null  float64
 8   final.output.tail_au                      18309 non-null  float64
 9   primary_cleaner.input.sulfate             18309 non-null  float64
 10  primary_cleaner.input.depressant       

### Due to the calculations we will be performing, columns not in a numerical format, as well as negative and zero values, can throw off calculations. Additionally numbers outside of normal ranges can significantly affect the model's ability to make accurate predictions. Therefore these extremes needed to be removed.

# Modeling the Data

## Linear Model 

In [20]:
# target
target = df2_cleaned['rougher.output.recovery']
features = df2_cleaned.drop(columns=['rougher.output.recovery'])

# split 
features_train, features_valid, target_train, target_valid = train_test_split(features, target, test_size=0.25, random_state=12345)

# model
model = LinearRegression()
model.fit(features_train, target_train)
predicted_valid = model.predict(features_valid)

# Mean Absolute Error (MAE)
mae = mean_absolute_error(target_valid, predicted_valid)
print('MAE:', mae)


MAE: 1.5365183740337998


## Decision Tree Model 

In [21]:
#  model
model = DecisionTreeRegressor(max_depth=None, random_state=12345)
model.fit(features_train, target_train)
predicted_valid = model.predict(features_valid)

# MAE
mae = mean_absolute_error(target_valid, predicted_valid)
print('MAE:', mae)

MAE: 0.9600036789104983


## Random Forest Model 

In [22]:
# model
model = RandomForestRegressor(n_estimators=100, max_features='sqrt', max_depth=None, random_state=12345)
model.fit(features_train, target_train)
predicted_valid = model.predict(features_valid)

# MAE
mae = mean_absolute_error(target_valid, predicted_valid)
print('MAE:', mae)

MAE: 1.1014355135182103


### We tested three different models to predict rougher output recovery. The Decision Tree Model was the only one to have an MAE below 1, so it's the best option to tune further.
### After applying cross validation, even after tuning parameter, it did not perform as well.  Most likely as issue with overfitting. So we went with a Gradient Boosting Regressor. 

## Tunning Our Model 

### Below is the code used to come up with the best Gradien Boosting model.  I have commented it out, you can run if you have about 15 minutes, or see the results below. 

In [ ]:
"""
# Parameter grid for GradientBoostingRegressor
param_grid = {
    'regressor__n_estimators': [100, 200, 300],
    'regressor__learning_rate': [0.01, 0.1, 0.2],
    'regressor__max_depth': [3, 4, 5, 6],
    'regressor__min_samples_split': [2, 5, 10],
    'regressor__min_samples_leaf': [1, 2, 4]}

# pipeline 
pipeline = Pipeline([('scaler', StandardScaler()), ('regressor', GradientBoostingRegressor(random_state=12345))])

# RandomizedSearchCV
search = RandomizedSearchCV(pipeline, param_distributions=param_grid, n_iter=20, scoring='neg_mean_absolute_error', n_jobs=-1, cv=5, random_state=12345, error_score='raise')
search.fit(features_train, target_train)

# model
best_model = search.best_estimator_
predicted_valid = best_model.predict(features_valid)

# MAE
mae = mean_absolute_error(target_valid, predicted_valid)
print('MAE:', mae)
print('Best Parameters:', search.best_params_)

# Cross-validation
cv_scores = cross_val_score(best_model, features_train, target_train, cv=5, scoring=make_scorer(mean_absolute_error, greater_is_better=False), n_jobs=-1)
cv_scores = -cv_scores  # Convert to positive MAE scores

print('Cross-Validated MAE scores:', cv_scores)
print('MAE:', cv_scores.mean())
"""


MAE: 0.4588986482038109
Best Parameters: {'regressor__n_estimators': 300, 'regressor__min_samples_split': 10, 'regressor__min_samples_leaf': 4, 'regressor__max_depth': 6, 'regressor__learning_rate': 0.1}
Cross-Validated MAE scores: [0.50982369 0.52715666 0.52465886 0.50683448 0.47944065]
MAE: 0.5095828691416241


###### MAE: 0.4646021856183499
###### Best Parameters: {'regressor__n_estimators': 300, 'regressor__min_samples_split': 10, 'regressor__min_samples_leaf': 4, 'regressor__max_depth': 6, 'regressor__learning_rate': 0.1}
###### Cross-Validated MAE scores: [0.50692897 0.53085616 0.5298506  0.51486003 0.48074974]
###### Mean Cross-Validated MAE: 0.5126490994726576

## Cross-Validation 

In [ ]:

# best parameters
best_params = {
    'n_estimators': 300,
    'learning_rate': 0.1,
    'max_depth': 6,
    'min_samples_split': 10,
    'min_samples_leaf': 4}

# model
model = GradientBoostingRegressor(**best_params, random_state=12345)
model.fit(features_train, target_train)
predicted_valid = model.predict(features_valid)

# MAE
mae = mean_absolute_error(target_valid, predicted_valid)
print('MAE:', mae)
mae_scorer = make_scorer(mean_absolute_error, greater_is_better=False)

# Cross-validation
cv_scores = cross_val_score(model, features_train, target_train, cv=5, scoring=mae_scorer, n_jobs=-1)
cv_scores = -cv_scores

print('Cross-Validated MAE scores:', cv_scores)
print('Mean Cross-Validated MAE:', cv_scores.mean())

MAE: 0.46399637898069745
Cross-Validated MAE scores: [0.51133051 0.51845972 0.52626707 0.50327064 0.48068619]
Mean Cross-Validated MAE: 0.5080028259121475


### When cross-validating our tuned model, all three folds as well as the average from them, closely follow our original MAE. This is a good indicator of the stability of the model across different data splits.

## Calculate Symmetric Mean Absolute Percentage Error (sMAPE)

In [25]:
#sMAPE fucntion 
def calculate_smape(actual, forecast):
    actual = np.array(actual)
    forecast = np.array(forecast)
    denominator = (np.abs(actual) + np.abs(forecast)) / 2
    diff = np.abs(forecast - actual)
    
    # Avoid division by zero
    valid_mask = denominator != 0
    smape = np.mean(diff[valid_mask] / denominator[valid_mask]) * 100
    return smape

# model
model = GradientBoostingRegressor(**best_params, random_state=12345)
model.fit(features_train, target_train)
predicted_valid = model.predict(features_valid)

# MAE
mae = mean_absolute_error(target_valid, predicted_valid)
print('MAE:', mae)

# Symmetric Mean Absolute Percentage Error (sMAPE)
smape_rougher = calculate_smape(target_valid, predicted_valid)
print('sMAPE:', smape_rougher)

MAE: 0.46399637898069745
sMAPE: 0.5802012033237249


###  A MAE and sMAPE of 0.5 are as close to ideal as we could expect. Our model, though laborious in calculations, is very accurate.

## Model Target of final.output.recovery

In [26]:
# target
target = df2_cleaned['final.output.recovery']
features = df2_cleaned.drop(columns=['final.output.recovery'])

# model
model = GradientBoostingRegressor(**best_params, random_state=12345)
model.fit(features_train, target_train)
predicted_valid = model.predict(features_valid)

# MAE
mae = mean_absolute_error(target_valid, predicted_valid)
print('MAE:', mae)

# sMAPE
smape_final = calculate_smape(target_valid, predicted_valid)
print('sMAPE:', smape_final)

MAE: 0.46399637898069745
sMAPE: 0.5802012033237249


In [27]:
# target
target_final = df2_cleaned['final.output.recovery']
features_final = df2_cleaned.drop(columns=['final.output.recovery'])

# split
features_train_final, features_valid_final, target_train_final, target_valid_final = train_test_split(features_final, target_final, test_size=0.25, random_state=12345)

# model
model = GradientBoostingRegressor(**best_params, random_state=12345)
model.fit(features_train_final, target_train_final)
predicted_valid_final = model.predict(features_valid_final)

# MAE
mae_final = mean_absolute_error(target_valid_final, predicted_valid_final)
print('MAE:', mae_final)

# sMAPE
smape_final = calculate_smape(target_valid_final, predicted_valid_final)
print('sMAPE:', smape_final)

MAE: 0.6610569821975548
sMAPE: 1.1171933875795652


In [28]:
finally_smape = .25*smape_rougher+.75*smape_final 
print('The Final sMAPE is:', finally_smape)

The Final sMAPE is: 0.9829453415156051


### Given how our Final sMAPE formula was calculated, sMAPE(final) is weighted 3:1. We could have improved our final calculation if we had chosen to tune our model originally with 'final.output.recovery' as the target. However, achieving only a 1% error and the fact that our model generalizes to other targets so well are huge wins.

All project files are available at Github : https://github.com/Tom-Kinstle/sprint_10